# Imports

In [ ]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
import contractions
import nltk
from tabulate import tabulate
from textblob import TextBlob
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
import plotly.graph_objects as go
from collections import Counter
from wordcloud import WordCloud
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import plotly.graph_objects as go
import re


nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('omw-1.4')

pyLDAvis.enable_notebook()

# Data Loading

## Loading data from Twitter (X)

In [ ]:
# Loading CSV files and concatenating them into a single DataFrame
data_twitter = pd.concat([
    pd.read_csv('filtered_super_app_2.csv'),
    pd.read_csv('filtered_super_apps_tweets.csv')
], ignore_index=True)

# Adding a new column called 'source' with the value 'super_apps_twitter'
data_twitter['source'] = 'super_apps_twitter'

In [ ]:
print(len(data_twitter))

In [ ]:
# detecting the language of the tweets and filtering out non-English tweets
import langid
data_twitter["language_det"] = data_twitter["Tweet_Content"].apply(lambda x: langid.classify(x)[0] if isinstance(x, str) else "")
data_twitter = data_twitter[data_twitter["language_det"] == "en"]
print(len(data_twitter))

In [ ]:
# Sometimes the superapp phrase is in the username, thats why we are filtering tweets that contain the phrase in the tweet content
# Defining the phrases to search for
phrases = ["super app", "superapp", "super-app", "super apps", "superapps", "super-apps"]

# Creating a regex pattern to match any of the phrases (case-insensitive)
pattern = '|'.join(map(re.escape, phrases))

# Filtering the dataset for tweets containing any of the phrases
data_twitter = data_twitter[data_twitter['Tweet_Content'].str.contains(pattern, case=False, na=False)]


In [ ]:
# Defining a function to categorize data based on conditions
def categorize(row):
    # Converting comments to lowercase for case-insensitive matching
    text = row['Tweet_Content'].lower()
    if 'wechat' in text:
        return 'wechat'
    elif 'kakaotalk' in text or 'kakao talk' in text or 'kakao-talk' in text:
        return 'kakaotalk'
    elif 'gojek' in text:
        return 'gojek'
    elif 'grab' in text:
        return 'grab'
    elif 'revolut' in text:
        return 'revolut'
    elif 'alipay' in text:
        return 'alipay'
    else:
        return 'other'  

# Applying the function to create the new 'category' column
data_twitter['category'] = data_twitter.apply(categorize, axis=1)

In [ ]:
data_twitter.head()

In [ ]:
print(len(data_twitter))

## Load data from Youtube

In [ ]:
dfs_youtube_1 = [
    pd.read_csv('YouTube Why Its so hard to.csv'),
    pd.read_csv('YouTube CNBC.csv'),
    pd.read_csv('YouTube Elon_failed.csv'),
    pd.read_csv('YouTube Why_elon.csv'),
    pd.read_csv('YouTube CNBC_what.csv'),
    pd.read_csv('YouTube How_china_is_changing.csv')
    ]
dfs_youtube_1 = pd.concat(dfs_youtube_1, ignore_index=True)

In [ ]:
dfs_revolut_1 = [
    pd.read_csv('YouTube truth_revolut.csv'),
    pd.read_csv('YouTube revolut_vs_wise.csv'),
]
dfs_grab_1 = [
    pd.read_csv('YouTube grab.csv'),
    pd.read_csv('YouTube revolut_+_grab.csv')
]
dfs_kakaotalk_1 = [
    pd.read_csv('YouTube KakaoTalk_how_to_find.csv'),
    pd.read_csv('YouTube KakaoTalk_signup.csv')
]
dfs_wechat_1 = [pd.read_csv('YouTube wechat_tencent.csv')]
dfs_alipay_1 = [
    pd.read_csv('YouTube alipay.csv'),
    pd.read_csv('YouTube alipay_2.csv')
]
dfs_gojek_1 = [
    pd.read_csv('YouTube gojek.csv')
]
dfs_gojek_1 = pd.concat(dfs_gojek_1, ignore_index=True)
dfs_wechat_1 = pd.concat(dfs_wechat_1, ignore_index=True)
dfs_alipay_1 = pd.concat(dfs_alipay_1, ignore_index=True)
dfs_kakaotalk_1 = pd.concat(dfs_kakaotalk_1, ignore_index=True)
dfs_grab_1 = pd.concat(dfs_grab_1, ignore_index=True)
dfs_revolut_1 = pd.concat(dfs_revolut_1, ignore_index=True)



In [ ]:
# Creating a new column called 'source' with certain value (for example: 'youtube_gojek') for all DataFrames in certain list (for example: 'dfs_gojek_1')
dfs_gojek_1['category'] = 'gojek'
dfs_wechat_1['category'] = 'wechat'
dfs_alipay_1['category'] = 'alipay'
dfs_kakaotalk_1['category'] = 'kakaotalk'
dfs_grab_1['category'] = 'grab'
dfs_revolut_1['category'] = 'revolut'


In [ ]:
# Defining a function to categorize data based on conditions
def categorize(row):
    # Converting the comment to lowercase for case-insensitive matching
    text = row['Comment'].lower()
    if 'wechat' in text:
        return 'wechat'
    elif 'kakaotalk' in text or 'kakao talk' in text or 'kakao-talk' in text:
        return 'kakaotalk'
    elif 'gojek' in text:
        return 'gojek'
    elif 'grab' in text:
        return 'grab'
    elif 'revolut' in text:
        return 'revolut'
    elif 'alipay' in text:
        return 'alipay'
    else:
        return 'other' 

# Applying the function to create the new 'category' column
dfs_youtube_1['category'] = dfs_youtube_1.apply(categorize, axis=1)

In [ ]:
# merging youtube data
data_youtube_1 = pd.concat([dfs_youtube_1, dfs_alipay_1, dfs_gojek_1, dfs_grab_1, dfs_kakaotalk_1, dfs_revolut_1, dfs_wechat_1], ignore_index=True)
data_youtube_1['source'] = 'super_apps_youtube'
data_youtube_1.head()
print(len(data_youtube_1))


In [ ]:
# detecting the language of the tweets and filtering out non-English tweets
data_youtube_1["language_det"] = data_youtube_1["Comment"].apply(lambda x: langid.classify(x)[0] if isinstance(x, str) else "")
data_youtube_1 = data_youtube_1[data_youtube_1["language_det"] == "en"]
print(len(data_youtube_1))

# Data preprocessing and exploration

## Column dropping + duplicates checking

### Dropping unwanted columns and renaming columns - Twitter

In [ ]:
data_twitter.head()

In [ ]:
# Verifying current columns first
print("Original columns in super_app:")
print(data_twitter.columns.tolist())


# Renaming specific columns
renamed_columns = {
    'UTC_Time': 'Date_of_comment',
    'Tweet_Content': 'Comment',
}

data_twitter = data_twitter.rename(columns=renamed_columns)

# Defining columns to remove (adjust based on actual column names)
columns_to_drop = [
    'Author_Web_Page_URL', 
    'Post_ID', 
    'Tweet_URL',
    'Language',
    'Replying_to',
    'Replying_to Whom',
    'Replying_to_Whom_URL',
    'Replying_to_Whom_Username',
    'Reply_to_Whom_Handle',
    'Replying_to',
    'Tweet_image_URL',
    'Bookmark_Count',
    'View_Count',
    'Reply_Count',
    'Repost_Count',
    'Ads',
    'Verified',
    'Author_Handle',
    'Post_URL',
    'Query_Str',
    'Verified_Status',
    'Like_Count',
    'Tweet_Image_URL',
    'Reply_to_Whom',
    'Reply_to_Whom_URL',
    'Reply_to_Whom_Username',
    'language_det'


]

# Safe removal - only drop existing columns
def safe_column_drop(df, columns):
    existing_cols = [col for col in columns if col in df.columns]
    return df.drop(existing_cols, axis=1)

# Applying the function to dataset
data_twitter = safe_column_drop(data_twitter, columns_to_drop)


# Verifying remaining columns
print("\nColumns after removal in super_app:")
print(data_twitter.columns.tolist())


data_twitter.head()


### Getting rid of duplicates in Twitter data

In [ ]:
import pandas as pd

def remove_duplicates(df):
    # Printing the original row count
    print(f"Original row count: {len(df)}")
    
    # Removing duplicates based on 'Tweet_Content'
    df_unique = df.drop_duplicates(subset='Comment', keep='first')
    
    # Printing new row count
    print(f"Row count after removing duplicates: {len(df_unique)}")
    
    # Calculating the number of duplicates removed
    duplicates_removed = len(df) - len(df_unique)
    print(f"Number of duplicates removed: {duplicates_removed}\n")
    
    return df_unique

# Processing the merged DataFrame
print("Processing merged dataset:")
data_twitter = remove_duplicates(data_twitter)




### Dropping unwanted columns and renaming columns - Youtube

In [ ]:
# Renaming specific columns
renamed_columns = {
    'Video Title': 'Title_of_video',
    'Author': 'Author_Name',
    'Date and Time': 'Date_of_comment',
    'Comment': 'Comment'
}

data_youtube_1 = data_youtube_1.rename(columns=renamed_columns)
data_youtube_1 = data_youtube_1.drop(columns=['Title_of_video', 'language_det'])


data_youtube_1.head()

### Getting rid of duplicates in YouTube data

In [ ]:
import pandas as pd

def remove_duplicates(df):
    # Printing the original row count
    print(f"Original row count: {len(df)}")
    
    # Removing duplicates based on 'Tweet_Content'
    df_unique = df.drop_duplicates(subset='Comment', keep='first')
    
    # Printing new row count
    print(f"Row count after removing duplicates: {len(df_unique)}")
    
    # Calculating the number of duplicates removed
    duplicates_removed = len(df) - len(df_unique)
    print(f"Number of duplicates removed: {duplicates_removed}\n")
    
    return df_unique

# Processing the merged DataFrame
print("Processing merged dataset:")
data_youtube_1 = remove_duplicates(data_youtube_1)




### Converting time to UTC format + editing the "Date_of_comment" column - YouTube

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import dateparser

# Assuming 'df' is your DataFrame

# Function to clean and convert dates
def convert_date(date_str):
    # Remove unwanted phrases
    date_str = date_str.replace("(upraveno)", "").replace("(edited)", "").strip()
    
    # Convert date string to datetime object
    try:
        dt = dateparser.parse(date_str)
    except Exception as e:
        print(f"Error parsing date: {date_str}. Error: {e}")
        return None
    
    if dt is None:
        # If dateparser fails, try manual conversion for "X months ago"
        if "months ago" in date_str:
            months_ago = int(date_str.split("months ago")[0])
            dt = datetime.now() - timedelta(days=months_ago*30)
        elif "months" in date_str:
            months_ago = int(date_str.split("months")[0])
            dt = datetime.now() - timedelta(days=months_ago*30)
        else:
            return None
    
    # Set date to the 15th of the month
    dt = dt.replace(day=15, hour=12, minute=0, second=0)
    
    # Convert to UTC timezone
    dt = dt.astimezone(tz=None)  # Assuming system timezone is UTC
    
    return dt.strftime('%Y-%m-%d %H:%M:%S%z')

# Apply the function to the 'comment_time' column
data_youtube_1['Date_of_comment'] = data_youtube_1['Date_of_comment'].apply(convert_date)
data_youtube_1.head()



## Merging data from Twitter and Youtube

In [ ]:
import pandas as pd

# Aligning columns
all_columns = set(data_twitter.columns) | set(data_youtube_1.columns)
data_twitter = data_twitter.reindex(columns=all_columns)
data_youtube_1 = data_youtube_1.reindex(columns=all_columns)

# Concatenating DataFrames
data = pd.concat([data_twitter, data_youtube_1], ignore_index=True)

# Saving the merged DataFrame to a new CSV file
data.head()



## Checking for duplicates in merged file

In [ ]:
import pandas as pd

def remove_duplicates(df):
    # Printing original row count
    print(f"Original row count: {len(df)}")
    
    # Removing duplicates based on 'Tweet_Content'
    df_unique = df.drop_duplicates(subset='Comment', keep='first')
    
    # Printing the new row count
    print(f"Row count after removing duplicates: {len(df_unique)}")
    
    # Calculating the number of duplicates removed
    duplicates_removed = len(df) - len(df_unique)
    print(f"Number of duplicates removed: {duplicates_removed}\n")
    
    return df_unique

# Processing the merged DataFrame
print("Processing merged dataset:")
clean_data = remove_duplicates(data)




## Checking and deleting missing values

In [ ]:
# Removing the row(s) if they are not important
data = data[data['Comment'].apply(lambda x: isinstance(x, str))]

# Checking for missing values
print(data['Comment'].isnull().sum())

# Checking for non-string values
print(data['Comment'].apply(type).value_counts())

### Checking the format of the Date_of_comment column

In [ ]:
# Ensure all data in Date_of_comment is in the same format
data['Date_of_comment'] = pd.to_datetime(data['Date_of_comment'], errors='coerce', utc=True)

# Check for any rows where the conversion failed
invalid_dates = data[data['Date_of_comment'].isnull()]
if not invalid_dates.empty:
    print(f"Found {len(invalid_dates)} rows with invalid dates. These rows will be removed.")
    data = data.dropna(subset=['Date_of_comment'])

# Confirm the format
print("All dates are now in the same format.")
print(data['Date_of_comment'].head())

## Text preprocessing

### Cleaning the text

In [ ]:
# Getting indexes of rows with only empty spaces in column body
empty_rows = data[data['Comment'].str.strip() == ''].index
print(empty_rows.tolist())

### Cleaning the "Title_of_video" column

In [ ]:
#Text cleaning function
def clean_title(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    text = re.sub(r'\s+', ' ', text) # Remove extra spaces
    return text

data['Title_of_video'] = data['Title_of_video'].astype(str).apply(clean_title)

### Cleaning the "Author_Name" column

In [ ]:
#Text cleaning function
def clean_author(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
#    text = ' '.join([contractions.fix(word) for word in text.split()]) # Expand contractions
#    text = text.lower()  # Convert to lowercase
#    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text) # Remove extra spaces
    return text

data['Author_Name'] = data['Author_Name'].astype(str).apply(clean_author)

### Cleaning the "Comment" column

In [ ]:
#Text cleaning function
def clean_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    text = ' '.join([contractions.fix(word) for word in text.split()]) # Expand contractions
    text = text.lower()  # Convert to lowercase        ???????????????
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text) # Remove extra spaces
    return text

# Applying the text functions
data['cleaned_body'] = data['Comment'].astype(str).apply(clean_text)


### Part of Speech Tagging and Lemmatization

In [ ]:
# POS function - to map tags to worndet tags used in lemmatization
def wordnet_tagger(nltk_tag):
    if nltk_tag.startswith("N"):
        return wordnet.NOUN
    elif nltk_tag.startswith("V"):
        return wordnet.VERB
    elif nltk_tag.startswith("R"):
        return wordnet.ADV
    elif nltk_tag.startswith("J"):
        return wordnet.ADJ
    else:
        return None

# To convert pos tags to wordnet tags
def lemmatize(text):

  	# Tokenize the text first
    tokens = word_tokenize(text)

    # Tagging part
    pos_text = nltk.pos_tag(tokens)
    tagged_text = []
    for word, tag in pos_text:
        tagged_text.append((word, wordnet_tagger(tag)))

    # Lemmatization part
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = []
    for word, tag in tagged_text:
        if tag is None:
            lemmatized_text.append(word)
        else:
            lemmatized_text.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_text

data['lemma'] = data['cleaned_body'].apply(lemmatize)

### Stopwords Removal

In [ ]:
# Stopwords removal
custom_stop_words = set(stopwords.words('english'))

data['tokens'] = data['lemma'].apply(lambda x: [word for word in x if word not in custom_stop_words])

### Dictionary Creation

In [ ]:
# Creating a Gensim dictionary and corpus
dictionary = Dictionary(data['tokens'])
corpus = [dictionary.doc2bow(text) for text in data['tokens']]

# Exploratory Data Analysis

## Timeline of used tweets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Converting UTC_Time to datetime if not already
data['Date_of_comment'] = pd.to_datetime(data['Date_of_comment'], utc=True)

# Sorting the merged dataframe by UTC_Time
data = data.sort_values('Date_of_comment')

# Finding the earliest and latest dates for the entire dataset
earliest_comment = data['Date_of_comment'].min()
latest_comment = data['Date_of_comment'].max()

# Finding the earliest and latest dates for Twitter
twitter_data = data[data['source'] == 'super_apps_twitter']
earliest_tweet = twitter_data['Date_of_comment'].min()
latest_tweet = twitter_data['Date_of_comment'].max()

# Finding the earliest and latest dates for YouTube
youtube_data = data[data['source'] == 'super_apps_youtube']
earliest_youtube_comment = youtube_data['Date_of_comment'].min()
latest_youtube_comment = youtube_data['Date_of_comment'].max()

# Creating events list
events = ['Earliest tweet', 'Latest tweet', 'Earliest YouTube comment', 'Latest YouTube comment']
dates = [earliest_tweet, latest_tweet, earliest_youtube_comment, latest_youtube_comment]

colors = ['skyblue', 'salmon', 'lightgreen', 'orange']
color_names = ['Sky Blue', 'Salmon', 'Light Green', 'Orange']

plt.figure(figsize=(14, 6))

# Plotting timeline for all events
plt.hlines(1, min(dates), max(dates), colors='lightgray', linestyles='dotted')

for i, (date, color) in enumerate(zip(dates, colors)):
    plt.scatter(date, 1, color=color, zorder=5, s=100)

plt.yticks([])  # Removing y-axis
plt.xlabel('Timeline')
plt.title('Timeline of "Super App(s)" Comments')

# Formatting dates for the table
formatted_dates = [date.strftime('%Y-%m-%d %H:%M:%S') for date in dates]

table_data = [[events[i], formatted_dates[i], color_names[i]] for i in range(len(events))]
column_labels = ['Event', 'Date', 'Color']
plt.table(cellText=table_data, colLabels=column_labels, loc='bottom', cellLoc='center', bbox=[0.1, -0.5, 0.8, 0.3])

plt.subplots_adjust(bottom=0.45)  # Leaving space for the table
plt.tight_layout()
plt.show()

# Printing exact dates and additional information
print("Exact dates:")
for event, date in zip(events, dates):
    print(f"{event}: {date}")

print(f"\nTotal number of comments: {len(data)}")
print(f"Number of 'super_apps' tweets: {len(twitter_data)}")
print(f"Number of 'super_apps' YouTube comments: {len(youtube_data)}")


## Temporal Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pytz

# Ensuring 'Date_of_comment' is in datetime format
data['Date_of_comment'] = pd.to_datetime(data['Date_of_comment'])

# Separating data by source
twitter_data = data[data['source'] == 'super_apps_twitter']
youtube_data = data[data['source'] == 'super_apps_youtube']

# Setting indices for both datasets
twitter_data = twitter_data.set_index('Date_of_comment', drop=False)
youtube_data = youtube_data.set_index('Date_of_comment', drop=False)

# Getting current date in UTC
current_date = datetime.datetime.now(pytz.utc)

# Truncation of both datasets
last_date = min(current_date, data['Date_of_comment'].max())
twitter_data = twitter_data[twitter_data.index <= last_date]
youtube_data = youtube_data[youtube_data.index <= last_date]

# Resampling frequencies for both sources
frequency_data = {
    'daily': {
        'twitter': twitter_data.resample('D').size(),
        'youtube': youtube_data.resample('D').size()
    },
    'weekly': {
        'twitter': twitter_data.resample('W').size(),
        'youtube': youtube_data.resample('W').size()
    },
    'monthly': {
        'twitter': twitter_data.resample('M').size(),
        'youtube': youtube_data.resample('M').size()
    },
    'yearly': {
        'twitter': twitter_data.resample('Y').size().fillna(0),
        'youtube': youtube_data.resample('Y').size().fillna(0)
    }
}

# Creating side-by-side plots for each timeframe
plt.figure(figsize=(20, 16))

# Daily frequencies - side by side
plt.subplot(4, 2, 1)
sns.lineplot(x=frequency_data['daily']['twitter'].index, y=frequency_data['daily']['twitter'].values, color='blue')
plt.title('Daily Twitter Activity')
plt.ylabel('Number of Tweets')

plt.subplot(4, 2, 2)
sns.lineplot(x=frequency_data['daily']['youtube'].index, y=frequency_data['daily']['youtube'].values, color='orange')
plt.title('Daily YouTube Activity')
plt.ylabel('Number of Comments')

# Weekly frequencies - side by side
plt.subplot(4, 2, 3)
sns.lineplot(x=frequency_data['weekly']['twitter'].index, y=frequency_data['weekly']['twitter'].values, color='blue')
plt.title('Weekly Twitter Activity')
plt.ylabel('Number of Tweets')

plt.subplot(4, 2, 4)
sns.lineplot(x=frequency_data['weekly']['youtube'].index, y=frequency_data['weekly']['youtube'].values, color='orange')
plt.title('Weekly YouTube Activity')
plt.ylabel('Number of Comments')

# Monthly frequencies - side by side
plt.subplot(4, 2, 5)
sns.lineplot(x=frequency_data['monthly']['twitter'].index, y=frequency_data['monthly']['twitter'].values, color='blue')
plt.title('Monthly Twitter Activity')
plt.ylabel('Number of Tweets')

plt.subplot(4, 2, 6)
sns.lineplot(x=frequency_data['monthly']['youtube'].index, y=frequency_data['monthly']['youtube'].values, color='orange')
plt.title('Monthly YouTube Activity')
plt.ylabel('Number of Comments')

# Yearly frequencies - side by side
plt.subplot(4, 2, 7)
# Converting indexes to year strings for twitter (X)
twitter_yearly = frequency_data['yearly']['twitter']
twitter_yearly.index = twitter_yearly.index.year.astype(str)
sns.lineplot(x=twitter_yearly.index, y=twitter_yearly.values, color='blue')
plt.title('Yearly Twitter Activity')
plt.ylabel('Number of Tweets')

plt.subplot(4, 2, 8)
# Converting indexes to year strings for youtube
youtube_yearly = frequency_data['yearly']['youtube']
youtube_yearly.index = youtube_yearly.index.year.astype(str)
sns.lineplot(x=youtube_yearly.index, y=youtube_yearly.values, color='orange')
plt.title('Yearly YouTube Activity')
plt.ylabel('Number of Comments')

plt.tight_layout()
plt.show()

# Printing statistics
print(f"Total number of comments/tweets: {len(data)}")
print(f"Twitter posts: {len(twitter_data)}")
print(f"YouTube comments: {len(youtube_data)}")
print(f"\nDate range: from {data.index.min()} to {data.index.max()}")
print(f"Twitter date range: {twitter_data.index.min()} to {twitter_data.index.max()}")
print(f"YouTube date range: {youtube_data.index.min()} to {youtube_data.index.max()}")

# Printing max activity statistics by platform
for platform in ['twitter', 'youtube']:
    label = "Tweets" if platform == 'twitter' else "Comments"
    print(f"\n{platform.capitalize()} Max Activity:")
    print(f"Day: {frequency_data['daily'][platform].idxmax()} ({frequency_data['daily'][platform].max()} {label})")
    print(f"Week: {frequency_data['weekly'][platform].idxmax()} ({frequency_data['weekly'][platform].max()} {label})")
    print(f"Month: {frequency_data['monthly'][platform].idxmax().strftime('%Y-%m')} ({frequency_data['monthly'][platform].max()} {label})")
    print(f"Year: {frequency_data['yearly'][platform].idxmax()} ({frequency_data['yearly'][platform].max()} {label})")


## User Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Counting tweets per user
user_tweet_counts = data['Author_Name'].value_counts()

# Top 10 users by tweet count
top_10_users = user_tweet_counts.head(10)

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(x=top_10_users.index, y=top_10_users.values)
plt.title('Top 10 Users by Tweet/Comment Count')
plt.xlabel('User')
plt.ylabel('Number of Tweets/Comments')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Printing statistics
print(f"Total number of unique users: {len(user_tweet_counts)}")
print(f"User with most tweets/comments: {user_tweet_counts.index[0]} ({user_tweet_counts.iloc[0]} tweets)")
print("\nTop 10 users by tweet/comment count:")
print(top_10_users)





In [ ]:
# Settings so that we can see the comments in full size
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_rows', None)    


### Checking users and their tweets/comments 

In [ ]:
# Filtering comments by author
life_platform_comments = data[data['Author_Name'] == 'Life Platform | Official']
print(len(life_platform_comments))

# Displaying comments
print(life_platform_comments[['Date_of_comment', 'Comment']])
print(len(data))

In [ ]:
# filtering comments by author
iron_wil_becker_comments = data[data['Author_Name'] == 'Iron Wil Becker']
print(len(iron_wil_becker_comments))

# Displaying comments
print(iron_wil_becker_comments[['Date_of_comment', 'Comment']])
print(len(data))

### Removing all comments/tweets from certain users

In [ ]:
# Filtering out posts from the user "Iron Wil Becker"
data = data[data['Author_Name'] != 'Iron Wil Becker']

# Verifying the deletion
print(f"Remaining posts: {len(data)}")

In [ ]:
# Filtering out posts from the user "Life Platform | Official"
data = data[data['Author_Name'] != 'Life Platform | Official']

# Verifying the deletion
print(f"Remaining posts: {len(data)}")

### Cleaned version of the User Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Counting tweets per user
user_tweet_counts = data['Author_Name'].value_counts()

# Top 10 users by tweet count
top_10_users = user_tweet_counts.head(10)

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(x=top_10_users.index, y=top_10_users.values)
plt.title('Top 10 Users by Tweet/Comment Count')
plt.xlabel('User')
plt.ylabel('Number of Tweets/Comments')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Printing statistics
print(f"Total number of unique users: {len(user_tweet_counts)}")
print(f"User with most tweets/comments: {user_tweet_counts.index[0]} ({user_tweet_counts.iloc[0]} tweets)")
print("\nTop 10 users by tweet/comment count:")
print(top_10_users)





## Hashtag Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

def extract_hashtags(text):
    if isinstance(text, str):
        return re.findall(r'#(\w+)', text.lower())
    else:
        return []

# Extracting hashtags from all tweets
all_hashtags = data['Comment'].apply(extract_hashtags).sum()

# Counting hashtag occurrences
hashtag_counts = Counter(all_hashtags)

# Removing 'superapp' and 'superapps' from the count
del hashtag_counts['superapp']
del hashtag_counts['superapps']

# Top 20 hashtags
top_20_hashtags = pd.DataFrame(hashtag_counts.most_common(20), columns=['Hashtag', 'Count'])

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(x='Count', y='Hashtag', data=top_20_hashtags)
plt.title('Top 20 Co-occurring Hashtags with "super app(s)"')
plt.xlabel('Count')
plt.ylabel('Hashtag')
plt.tight_layout()
plt.show()

# Printing  statistics
print(f"Total number of hashtags used: {len(all_hashtags)}")
print(f"Number of unique hashtags: {len(hashtag_counts)}")
print("\nTop 20 co-occurring hashtags:")
print(top_20_hashtags)

# Hashtag co-occurrence
def get_hashtag_pairs(hashtags):
    return [(a, b) for idx, a in enumerate(hashtags) for b in hashtags[idx + 1:]]

hashtag_pairs = data['Comment'].apply(lambda x: get_hashtag_pairs(extract_hashtags(x))).sum()
pair_counts = Counter(hashtag_pairs)

print("\nTop 10 co-occurring hashtag pairs:")
for pair, count in pair_counts.most_common(10):
    print(f"{pair}: {count}")


The last four rows in the output represent co-occurring hashtag pairs. These are pairs of hashtags that appear together in the same tweet.

# Sentiment Analysis (Bert)

## Initialization of the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

# BERT-specific preprocessing
def bert_preprocess(text):
    """Minimal preprocessing optimized for BERT models"""
    text = str(text) if pd.notnull(text) else ""
    text = text.replace('\n', ' ')    # Remove newlines
    text = text.strip()               # Remove leading/trailing whitespace
    return text[:512]                 # Truncate to BERT's max length

# Using Twitter-optimized BERT model
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"  # for English tweets

# Initializing components with error handling
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# Configuring pipeline with batch processing
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,  
    truncation=True,
    max_length=512,
    batch_size=16 
)

def analyze_tweets(texts):
    """Process tweets in batches with error handling"""
    results = []
    for batch in tqdm([texts[i:i+16] for i in range(0, len(texts), 16)]):
        try:
            batch_results = sentiment_analyzer(batch)
            results.extend([result['label'] for result in batch_results])
        except Exception as e:
            print(f"Error processing batch: {e}")
            results.extend(['neutral']*len(batch))  # Fallback for failed batches
    return results

# Preprocessing and analyzing
data['clean_text_bert'] = data['Comment'].apply(bert_preprocess)
valid_texts = data[data['clean_text_bert'] != '']['clean_text_bert'].tolist()

print(f"Analyzing {len(valid_texts)} tweets...")
sentiments = analyze_tweets(valid_texts)

# Mapping predictions to standard labels
label_map = {
    'LABEL_0': 'negative',
    'LABEL_1': 'neutral',
    'LABEL_2': 'positive'
}

data['sentiment'] = 'neutral'  # Default value
data.loc[data['clean_text_bert'] != '', 'sentiment'] = [label_map.get(s, 'neutral') for s in sentiments]




## Visualization of sentiment based on superapp

In [ ]:
import plotly.graph_objs as go
import pandas as pd
import plotly.io as pio
pio.renderers.default = 'notebook'


# Getting unique categories
categories = sorted(data['category'].unique())

fig = go.Figure()

# Adding traces for each category
for category in categories:
    filtered_data = data[data['category'] == category]
    sentiment_counts = filtered_data['sentiment'].value_counts()

    # Calculating percentage
    total_comments = sentiment_counts.sum()
    sentiment_percentages = (sentiment_counts / total_comments) * 100

    # Adding trace with percentage labels
    fig.add_trace(go.Bar(
        x=sentiment_counts.index,
        y=sentiment_counts.values,
        name=str(category),
        visible=False,
        text=[f'{p:.2f}%' for p in sentiment_percentages],
        textposition='inside',
        hovertext=[f'{count} comments\n({p:.2f}%)' for count, p in zip(sentiment_counts, sentiment_percentages)],
        hoverinfo='text'
    ))

# Adding trace for all categories
all_sentiment_counts = data['sentiment'].value_counts()
total_all_comments = all_sentiment_counts.sum()
all_sentiment_percentages = (all_sentiment_counts / total_all_comments) * 100
fig.add_trace(go.Bar(
    x=all_sentiment_counts.index,
    y=all_sentiment_counts.values,
    name="All Categories",
    visible=True,
    text=[f'{p:.2f}%' for p in all_sentiment_percentages],
    textposition='inside',
    hovertext=[f'{count} comments\n({p:.2f}%)' for count, p in zip(all_sentiment_counts, all_sentiment_percentages)],
    hoverinfo='text'
))

buttons = []

# Adding a button for each category
for i, category in enumerate(categories):
    button = dict(
        label=str(category),
        method="update",
        args=[
            {"visible": [j == i for j in range(len(categories))] + [False]},
            {"title": f"Sentiment Distribution (Category: {category})"}
        ]
    )
    buttons.append(button)

# Adding button to show all categories
buttons.append(
    dict(
        label="All Categories",
        method="update",
        args=[
            {"visible": [False] * len(categories) + [True]},
            {"title": "Sentiment Distribution (All Categories)"}
        ]
    )
)

# Updating layout with buttons
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        buttons=buttons,
        direction="down",
        showactive=True,
        x=1.1,
        xanchor="right",
        y=1.15,
        yanchor="top"
    )],
    title="Sentiment Distribution (All Categories)",
    xaxis_title="Sentiment",
    yaxis_title="Number of Comments",
    barmode="group"
)

# Show the figure
fig.show()


## Visualization of sentiment based on the platform (YouTube or Twitter)

In [ ]:
import plotly.graph_objs as go
import pandas as pd
pio.renderers.default = 'notebook'


# Getting unique categories
sources = sorted(data['source'].unique())

fig = go.Figure()

# Adding traces for each category
for source in sources:
    filtered_data = data[data['source'] == source]
    sentiment_counts = filtered_data['sentiment'].value_counts()

    # Calculating percentage
    total_comments = sentiment_counts.sum()
    sentiment_percentages = (sentiment_counts / total_comments) * 100

    # Adding trace with percentage labels
    fig.add_trace(go.Bar(
        x=sentiment_counts.index,
        y=sentiment_counts.values,
        name=str(source),
        visible=False,
        text=[f'{p:.2f}%' for p in sentiment_percentages],
        textposition='inside',
        hovertext=[f'{count} comments\n({p:.2f}%)' for count, p in zip(sentiment_counts, sentiment_percentages)],
        hoverinfo='text'
    ))

# Adding a trace for all categories
all_sentiment_counts = data['sentiment'].value_counts()
total_all_comments = all_sentiment_counts.sum()
all_sentiment_percentages = (all_sentiment_counts / total_all_comments) * 100
fig.add_trace(go.Bar(
    x=all_sentiment_counts.index,
    y=all_sentiment_counts.values,
    name="All Categories",
    visible=True,
    text=[f'{p:.2f}%' for p in all_sentiment_percentages],
    textposition='inside',
    hovertext=[f'{count} comments\n({p:.2f}%)' for count, p in zip(all_sentiment_counts, all_sentiment_percentages)],
    hoverinfo='text'
))

buttons = []

# Adding a button for each category
for i, source in enumerate(sources):
    button = dict(
        label=str(source),
        method="update",
        args=[
            {"visible": [j == i for j in range(len(sources))] + [False]},
            {"title": f"Sentiment Distribution (Category: {source})"}
        ]
    )
    buttons.append(button)

# Adding button to show all categories
buttons.append(
    dict(
        label="All Sources",
        method="update",
        args=[
            {"visible": [False] * len(sources) + [True]},
            {"title": "Sentiment Distribution (All Categories)"}
        ]
    )
)

# Updating layout with buttons
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        buttons=buttons,
        direction="down",
        showactive=True,
        x=1.1,
        xanchor="right",
        y=1.15,
        yanchor="top"
    )],
    title="Sentiment Distribution (All Sources)",
    xaxis_title="Sentiment",
    yaxis_title="Number of Comments",
    barmode="group"
)

# Show the figure
fig.show()


## Sentiment Frequency Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def sentiment_breakdown(df, granularity='monthly'):
    # Color configuration
    color_map = {
        'positive': '#4CAF50',  # Green
        'neutral': '#2196F3',   # Blue
        'negative': '#F44336'   # Red
    }
    
    # Verify Date_of_comment exists
    if 'Date_of_comment' not in df.columns:
        if 'Date_of_comment' in df.index.names:
            df = df.reset_index()
        else:
            raise ValueError("Date_of_comment column not found in DataFrame")

    breakdown = df['sentiment'].value_counts(normalize=True) * 100
    print(f"\nSentiment Distribution:\n{breakdown}")
    
    # Enhanced temporal analysis
    df['date'] = pd.to_datetime(df['Date_of_comment'])
    
    if granularity == 'weekly':
        df['date'] = df['date'].dt.to_period('W').dt.start_time
    elif granularity == 'monthly':
        df['date'] = df['date'].dt.to_period('M').dt.start_time
    else:  # default to daily
        df['date'] = df['date'].dt.floor('D')
    
    # Sentiment over time
    plt.figure(figsize=(14, 7))
    for sentiment in ['positive', 'neutral', 'negative']:
        subset = df[df['sentiment'] == sentiment]
        counts = subset.groupby('date').size()
        if granularity == 'daily':
            counts.rolling(7).mean().plot(label=sentiment, alpha=0.8, color=color_map[sentiment])
        elif granularity == 'weekly':
            counts.rolling(4).mean().plot(label=sentiment, alpha=0.8, color=color_map[sentiment])
        elif granularity == 'monthly':
            counts.plot(label=sentiment, alpha=0.8, color=color_map[sentiment])  # Direct plot for monthly data
    
    plt.title(f'Sentiment Trends ({granularity.capitalize()} Analysis)')
    plt.ylabel(f'Comment Count ({granularity})')  # Updated label for monthly data
    plt.legend()
    plt.show()

    # Sentiment by hour of day
    df['hour'] = pd.to_datetime(df['Date_of_comment']).dt.hour
    hourly_sentiment = df.groupby(['hour', 'sentiment']).size().unstack()
    
    hourly_sentiment = hourly_sentiment[['positive', 'neutral', 'negative']]
    
    hourly_sentiment.plot(
        kind='bar', 
        stacked=True, 
        figsize=(12,6),
        color=[color_map[col] for col in hourly_sentiment.columns]
    )
    plt.title('Sentiment Distribution by Hour of Day')
    plt.xlabel('Hour (UTC)')
    plt.ylabel('Number of comments')
    plt.show()


data['Date_of_comment'] = pd.to_datetime(data['Date_of_comment'])
sentiment_breakdown(data.copy(), granularity='monthly')


# Topic Modelling

## Analysis of the most frequent words

### WordClouds - Most Frequent Words – Overall

In [ ]:
# Wordcloud for the whole dataset
# Word frequency
word_freq = Counter()
for post in data['tokens']:
    word_freq.update(post)

# Generating the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

# Displaying the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Most Frequent Words Overall')
plt.axis('off')
plt.show()


### WordClouds - Most Frequent Words – with few exclusions

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Defining words to exclude
exclude_words = {"super", "app", "superapp", "super app", "super-app", "super apps", "superapps", "apps", "super-apps", "via", "us", 
"get", "make", "use", "like", "new", "need", "go", "create", "become", "take", "platform"}

# Word frequency
word_freq_1 = Counter()
for post in data['tokens']:
    filtered_tokens = [word for word in post if word.lower() not in exclude_words]
    word_freq_1.update(filtered_tokens)

# Generating the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq_1)
# Displaying the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Most Frequent Words with few exclusions')
plt.axis('off')
plt.show()


### Table – Top 20 most used words – Overall

In [ ]:
# Top 20 most frequent words
# Sorting word frequencies in descending order and get the top 20
top_20_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:20]

# Generating the order with correct suffixes
def get_ordinal_suffix(n):
    if 10 <= n % 100 <= 20: 
        return f'{n}th'
    else:
        suffixes = {1: 'st', 2: 'nd', 3: 'rd'}
        return f'{n}{suffixes.get(n % 10, "th")}'

# Generating the "Place" row with suffixes
order_with_suffix = [get_ordinal_suffix(i + 1) for i in range(len(top_20_word_freq))]

# Preparing the table data
table_data = [['Place', *order_with_suffix]]  # First row with place labels
table_data.append(['Word'] + [word for word, freq in top_20_word_freq])  # Second row with words
table_data.append(['Frequency'] + [freq for word, freq in top_20_word_freq])  # Third row with frequencies

# Printing the transposed table
print('Top 20 Most Common Words (Overall):')
print(tabulate(table_data, headers='firstrow', tablefmt='grid'))


### Table – Top 20 most used words – with few exclusions

In [ ]:
# Top 20 with excluded words
# Words to exclude from the table
exclude_words = {"super", "app", "superapp", "super app", "super-app", "super apps", "superapps", "apps", "super-apps", 
"via", "us", "get", "make", "use", "like", "new", "need", "go", "create", "become", "take", "would", "know", "think", "want", "say", "see", "even"}
# Sorting the original word frequencies by frequency in descending order
sorted_word_freq = sorted(word_freq_1.items(), key=lambda x: x[1], reverse=True)

# Filtering out the words to exclude
filtered_sorted_word_freq = [(word, freq) for word, freq in sorted_word_freq if word not in exclude_words]

# Taking the top 20 after filtering
top_20_filtered_word_freq = filtered_sorted_word_freq[:20]

# Generating the order with correct suffixes for the top 20
order_with_suffix_filtered = [get_ordinal_suffix(i + 1) for i in range(len(top_20_filtered_word_freq))]

# Preparing the table data
filtered_table_data = [['Place', *order_with_suffix_filtered]]  # First row with place labels
filtered_table_data.append(['Word'] + [word for word, freq in top_20_filtered_word_freq])  # Second row with words
filtered_table_data.append(['Frequency'] + [freq for word, freq in top_20_filtered_word_freq])  # Third row with frequencies

# Printing the filtered table
print('Top 20 Most Common Words (Filtered):')
print(tabulate(filtered_table_data, headers='firstrow', tablefmt='grid'))

### WordClouds - Most Frequent Words related to Wechat, AliPay and KakaoTalk

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

# List of words to exclude (stop words)
stop_words = ["super", "app", "superapp", "super app", "super-app", "super apps", "superapps", "apps", "super-apps", "via", "us", "get", "make", "use", "like", "new", "need", "go", "create", "become", "take", "would", "know", "think", "want", "say", "see", "even"]

# Combining all comments for each party and preprocess text
party_comments = {}
for category, party in zip(["wechat", "kakaotalk", "alipay"], ["WeChat", "KakaoTalk", "AliPay"]):
    comments = ' '.join([' '.join(map(str, tokens)) for tokens in data[data['category'] == category]['tokens']])
    
    # Converting to lowercase and remove punctuation
    comments = comments.lower()
    comments = re.sub(r'[^\w\s]', '', comments)
    
    # Removing stop words
    words = comments.split()
    filtered_words = [word for word in words if word not in stop_words]
    filtered_comments = ' '.join(filtered_words)
    
    party_comments[party] = filtered_comments

# Generating word clouds
wordclouds = {}
for party, comments in party_comments.items():
    wordclouds[party] = WordCloud(width=800, height=400, background_color='white').generate(comments)

# Plotting the word clouds
plt.figure(figsize=(16, 8))
for i, (party, wordcloud) in enumerate(wordclouds.items(), 1):
    plt.subplot(1, len(wordclouds), i)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Most Common Words Associated with {party} platform')
    plt.axis('off')

plt.tight_layout()
plt.show()


### Tables - Most Frequent Words related to Wechat, AliPay and KakaoTalk

In [ ]:
import pandas as pd
from collections import Counter
from tabulate import tabulate

# Function to get ordinal suffix (e.g., 1st, 2nd, 3rd)
def get_ordinal_suffix(n):
    if 11 <= n % 100 <= 13:
        return f"{n}th"
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
        return f"{n}{suffix}"

# List of words to exclude (stop words)
stop_words = {"super", "app", "superapp", "super app", "super-app", "super apps", "superapps", "apps", "super-apps", "via", "us", "get", "make", "use", "like", "new", "need", "go", "create", "become", "take", "would", "know", "think", "want", "say", "see", "even"}

# Processing each category and generating tables
for category in ["wechat", "kakaotalk", "alipay"]:
    # Filtering data for the current category
    category_data = data[data['category'] == category]
    
    # Combining all tokens for the category
    all_tokens = [token for tokens_list in category_data['tokens'] for token in tokens_list]
    
    # Filtering out stop words
    filtered_tokens = [token for token in all_tokens if token not in stop_words]
    
    # Calculating word frequencies
    word_freq = Counter(filtered_tokens)
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)  # Sorting by frequency
    
    # Taking the top 20 words
    top_20_word_freq = sorted_word_freq[:20]
    
    # Generating the order with correct suffixes for the top 20
    order_with_suffix = [get_ordinal_suffix(i + 1) for i in range(len(top_20_word_freq))]
    
    # Preparing the table data
    table_data = [['Place', *order_with_suffix]]  # First row with place labels
    table_data.append(['Word'] + [word for word, freq in top_20_word_freq])  # Second row with words
    table_data.append(['Frequency'] + [freq for word, freq in top_20_word_freq])  # Third row with frequencies
    
    # Printing the table
    print(f'\nTop 20 Most Common Words for {category.capitalize()}:')
    print(tabulate(table_data, headers='firstrow', tablefmt='grid'))


### WordClouds - Most Frequent Words related to Revolut, Grab and Gojek

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

# List of words to exclude (stop words)
stop_words = ["super", "app", "superapp", "super app", "super-app", "super apps", "superapps", "apps", "super-apps", "via", "us", "get", "make", "use", "like", "new", "need", "go", "create", "become", "take", "would", "know", "think", "want", "say", "see", "even"]

# Combining all comments for each party and preprocessing text
party_comments = {}
for category, party in zip(["revolut", "grab", "gojek"], ["Revolut", "Grab", "Gojek"]):
    comments = ' '.join([' '.join(map(str, tokens)) for tokens in data[data['category'] == category]['tokens']])
    
    # Converting to lowercase and removing punctuation
    comments = comments.lower()
    comments = re.sub(r'[^\w\s]', '', comments)
    
    # Removing stop words
    words = comments.split()
    filtered_words = [word for word in words if word not in stop_words]
    filtered_comments = ' '.join(filtered_words)
    
    party_comments[party] = filtered_comments

# Generating word clouds
wordclouds = {}
for party, comments in party_comments.items():
    wordclouds[party] = WordCloud(width=800, height=400, background_color='white').generate(comments)

# Plotting the word clouds
plt.figure(figsize=(16, 8))
for i, (party, wordcloud) in enumerate(wordclouds.items(), 1):
    plt.subplot(1, len(wordclouds), i)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Most Common Words Associated with {party} platform')
    plt.axis('off')

plt.tight_layout()
plt.show()


### Tables - Most Frequent Words related to Revolut, Grab and Gojek

In [ ]:
import pandas as pd
from collections import Counter
from tabulate import tabulate

# Function to get ordinal suffix (e.g., 1st, 2nd, 3rd)
def get_ordinal_suffix(n):
    if 11 <= n % 100 <= 13:
        return f"{n}th"
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
        return f"{n}{suffix}"

# List of words to exclude (stop words)
stop_words = {"super", "app", "superapp", "super app", "super-app", "super apps", "superapps", "apps", "super-apps", "via", "us", "get", "make", "use", "like", "new", "need", "go", "create", "become", "take", "would", "know", "think", "want", "say", "see", "even"}

# Processing each category and generating tables
for category in ["grab", "gojek", "revolut"]:
    # Filtering data for the current category
    category_data = data[data['category'] == category]
    
    # Combining all tokens for the category
    all_tokens = [token for tokens_list in category_data['tokens'] for token in tokens_list]
    
    # Filtering out stop words
    filtered_tokens = [token for token in all_tokens if token not in stop_words]
    
    # Calculating word frequencies
    word_freq = Counter(filtered_tokens)
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)  # Sort by frequency
    
    # Taking the top 20 words
    top_20_word_freq = sorted_word_freq[:20]
    
    # Generating the order with correct suffixes for the top 20
    order_with_suffix = [get_ordinal_suffix(i + 1) for i in range(len(top_20_word_freq))]
    
    # Preparing the table data
    table_data = [['Place', *order_with_suffix]]  # First row with place labels
    table_data.append(['Word'] + [word for word, freq in top_20_word_freq])  # Second row with words
    table_data.append(['Frequency'] + [freq for word, freq in top_20_word_freq])  # Third row with frequencies
    
    # Printing the table
    print(f'\nTop 20 Most Common Words for {category.capitalize()}:')
    print(tabulate(table_data, headers='firstrow', tablefmt='grid'))


### Tables - Most Frequent Words on YouTube and Twitter (X)

In [ ]:
subset_data_youtube = data[data['source'].str.contains('super_apps_youtube', case=False, na=False)]
subset_data_twitter = data[data['source'].str.contains('super_apps_youtube', case=False, na=False)]

In [ ]:
import pandas as pd
from collections import Counter
from tabulate import tabulate

# Function to get ordinal suffix (e.g., 1st, 2nd, 3rd)
def get_ordinal_suffix(n):
    if 11 <= n % 100 <= 13:
        return f"{n}th"
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
        return f"{n}{suffix}"

# List of words to exclude (stop words)
stop_words = {"super", "app", "superapp", "super app", "super-app", "super apps", "superapps", "apps", "super-apps", "via", "us", "get", "make", "use", "like", "new", "need", "go", "create", "become", "take", "would", "know", "think", "want", "say", "see", "even"}

# Processing each category and generate tables
for source in ["super_apps_youtube", "super_apps_twitter"]:
    # Filtering data for the current category
    source_data = data[data['source'] == source]
    
    # Combining all tokens for the category
    all_tokens = [token for tokens_list in source_data['tokens'] for token in tokens_list]
    
    # Filtering out stop words
    filtered_tokens = [token for token in all_tokens if token not in stop_words]
    
    # Calculating word frequencies
    word_freq = Counter(filtered_tokens)
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)  # Sorting by frequency
    
    # Taking the top 20 words
    top_20_word_freq = sorted_word_freq[:20]
    
    # Generating the order with correct suffixes for the top 20
    order_with_suffix = [get_ordinal_suffix(i + 1) for i in range(len(top_20_word_freq))]
    
    # Preparing the table data
    table_data = [['Place', *order_with_suffix]]  # First row with place labels
    table_data.append(['Word'] + [word for word, freq in top_20_word_freq])  # Second row with words
    table_data.append(['Frequency'] + [freq for word, freq in top_20_word_freq])  # Third row with frequencies
    
    # Printing the table
    print(f'\nTop 20 Most Common Words for {source.capitalize()}:')
    print(tabulate(table_data, headers='firstrow', tablefmt='grid'))


## Analysis of the most frequent bigrams

### Most frequent bigrams

In [ ]:
from nltk import ngrams
from collections import Counter
import matplotlib.pyplot as plt

# Defining words to omit
omit_words = {"super", "app", "apps", "in", "of", "is", "on", "with", "for", "at"} 

# Combining all tokens into a single string
text = ' '.join([' '.join(tokens) for tokens in data['tokens']])

# Filtering out unwanted words
filtered_tokens = [word for word in text.split() if word.lower() not in omit_words]

# Generating bigrams
n = 2
grams = ngrams(filtered_tokens, n)
gram_counts = Counter(grams)

# Getting the most common n-grams
top_n_grams = gram_counts.most_common(20)

# Preparing data for plotting
labels, values = zip(*top_n_grams)
labels = [' '.join(gram) for gram in labels]

# Plotting bigrams
plt.figure(figsize=(10, 6))
plt.bar(labels, values)
plt.xticks(rotation=45, ha='right')
plt.xlabel('N-grams')
plt.ylabel('Frequency')
plt.title(f'Top 20 Most Frequent {n}-grams (Filtered)')
plt.tight_layout()
plt.show()

# Displaying as table using pandas
bigram_table = pd.DataFrame(top_n_grams, columns=['Bigram', 'Frequency'])
bigram_table['Bigram'] = bigram_table['Bigram'].apply(lambda x: ' '.join(x))  # Converting tuple to string
print("\nTop 20 Most Frequent Bigrams:\n")
print(bigram_table.to_string(index=False))


## Topic Modelling using BERTopic

### Topic Modelling – YouTube data

In [ ]:
subset_data_youtube = data[data['source'].str.contains('super_apps_youtube', case=False, na=False)]


In [ ]:
print(len(subset_data_youtube))

#### Model initialization – YouTube

In [ ]:
import plotly.io as pio
pio.renderers.default='notebook_connected'

import pandas as pd
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
import re
import nltk
from nltk.corpus import stopwords

nltk_stopwords = set(stopwords.words('english'))



extra_stopwords = ['the', 'to', 'and', 'is', 'of', 'in', 'for', 'app', 'apps', 'platform', 'superapp', 'superapps' ,'super','it', 'on', 'this', 'that', 'you', 'your', 'with', 'are', 'we']
all_stop_words = nltk_stopwords.union(set(ENGLISH_STOP_WORDS))

# Data Cleaning Function

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in all_stop_words]) 
    return text.lower()


# Loading and Cleaning Data
subset_data_youtube['cleaned_body'] = subset_data_youtube['Comment'].apply(clean_text)


vectorizer_model = CountVectorizer(stop_words=list(all_stop_words))

# Model Configuration

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

umap_model = UMAP(
    n_neighbors=15, 
    n_components=8,  
    min_dist=0.1,   
    metric='cosine',
    random_state=42
)


hdbscan_model = HDBSCAN(
    min_cluster_size=50, 
    min_samples=15,        
    cluster_selection_epsilon=0.1,
    prediction_data=True,
    metric='euclidean'
)


from bertopic.representation import KeyBERTInspired
representation_model = KeyBERTInspired()

from bertopic.representation import MaximalMarginalRelevance
representation_model = MaximalMarginalRelevance(diversity=0.5)


# BERTopic Model Initialization and Fitting
topic_model_youtube = BERTopic(
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,
    min_topic_size=150,
    nr_topics="auto",
    low_memory=True,
    calculate_probabilities=True,
    verbose=True
)

# Fitting the model
topics, probs = topic_model_youtube.fit_transform(subset_data_youtube['cleaned_body'].tolist())



# Additional outlier reduction
if -1 in topics:
    print("Reducing outliers...")
    new_topics = topic_model_youtube.reduce_outliers(subset_data_youtube['cleaned_body'].tolist(), topics, strategy="c-tf-idf", threshold=0.05)
    new_topics = topic_model_youtube.reduce_outliers(subset_data_youtube['cleaned_body'].tolist(), new_topics, strategy="distributions", threshold=0.02)
    topic_model_youtube.update_topics(subset_data_youtube['cleaned_body'].tolist(), topics=new_topics)



# Visualizations
topic_model_youtube.visualize_topics(width=1000, height=600).show()
topic_model_youtube.visualize_barchart(top_n_topics=10).show()

topic_info_youtube = topic_model_youtube.get_topic_info()
print(topic_info_youtube)

# Printing representative docs for top 5 topics
for topic in topic_info_youtube['Topic'][:5]:
    if topic != -1:
        print(f"\nTopic {topic} Representative Documents:")
        rep_docs = topic_model_youtube.get_representative_docs(topic)
        for doc in rep_docs[:3]:
            print(doc)
        print("-" * 50)

#### Getting detailed information about gotten topics – YouTube

In [ ]:
from bertopic import BERTopic
import pandas as pd

# Getting topic information
topic_info_youtube = topic_model_youtube.get_topic_info()

# Getting more details for each topic
topic_details_youtube = []
for topic in topic_info_youtube['Topic']:
    if topic != -1:  # Excluding the outlier topic
        words = topic_model_youtube.get_topic(topic)
        docs = topic_model_youtube.get_representative_docs(topic)
        topic_details_youtube.append({
            'Topic': topic,
            'Name': topic_info_youtube[topic_info_youtube['Topic'] == topic]['Name'].values[0],
            'Count': topic_info_youtube[topic_info_youtube['Topic'] == topic]['Count'].values[0],
            'Top Words': ', '.join([word for word, _ in words[:10]]),
            'Representative Docs': docs[:3]  # Getting the first 3 representative documents
        })

# Converting to DataFrame for easier viewing
df_topic_details_youtube = pd.DataFrame(topic_details_youtube)
print(df_topic_details_youtube.to_string())


### Topic Modelling – Twitter (X) data

In [ ]:
subset_data_twitter = data[data['source'].str.contains('super_apps_twitter', case=False, na=False)]


In [ ]:
print(len(subset_data_twitter))

#### Model initialization – Twitter (X)

In [ ]:
import plotly.io as pio
pio.renderers.default='notebook_connected'

import pandas as pd
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
import re

# Data Cleaning Function
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.lower()

# Loading and Cleaning Data
subset_data_twitter['cleaned_body'] = subset_data_twitter['Comment'].apply(clean_text)

# Custom Stopwords
custom_stopwords = [
    "superapp", "apps", "app", "services", "platform", "digital", "social", "thank", "good", "like", "south", 
    "just", "hey", "oh", "really", "little", "im", "dont", "youre", "hes", "shes", 
    "theyre", "its", "super", "one", "com"
]
all_stop_words = set(ENGLISH_STOP_WORDS)#.union(set(custom_stopwords))

# Model Configuration
vectorizer_model = CountVectorizer(stop_words=list(all_stop_words))
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

umap_model = UMAP(
    n_neighbors=15,
    n_components=10,
    min_dist=0.05,
    metric='cosine',
    random_state=42
)

hdbscan_model = HDBSCAN(
    min_cluster_size=50,
    min_samples=15,
    cluster_selection_epsilon=0.1,
    prediction_data=True,
    metric='euclidean'
)

from bertopic.representation import KeyBERTInspired
representation_model = KeyBERTInspired()

# BERTopic Model Initialization and Fitting
topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,
    min_topic_size=100, 
    nr_topics="auto",
    low_memory=True,
    calculate_probabilities=True,
    verbose=True
)

# Fitting the model
topics, probs = topic_model.fit_transform(subset_data_twitter['cleaned_body'].tolist())

# Additional outlier reduction
if -1 in topics:
    print("Reducing outliers...")
    new_topics = topic_model.reduce_outliers(subset_data_twitter['cleaned_body'].tolist(), topics, strategy="c-tf-idf", threshold=0.1)
    new_topics = topic_model.reduce_outliers(subset_data_twitter['cleaned_body'].tolist(), new_topics, strategy="distributions", threshold=0.01)
    topic_model.update_topics(subset_data_twitter['cleaned_body'].tolist(), topics=new_topics)



# Visualizations
topic_model.visualize_topics(width=1000, height=600).show()
topic_model.visualize_barchart(top_n_topics=10).show()

topic_info = topic_model.get_topic_info()
print(topic_info)

# Printing representative docs for top 5 topics
for topic in topic_info['Topic'][:5]:
    if topic != -1:
        print(f"\nTopic {topic} Representative Documents:")
        rep_docs = topic_model.get_representative_docs(topic)
        for doc in rep_docs[:3]:
            print(doc)
        print("-" * 50)

#### Getting detailed information about gotten topics – Twitter (X)

In [ ]:
from bertopic import BERTopic
import pandas as pd

# Getting topic information
topic_info = topic_model.get_topic_info()

# Getting more details for each topic
topic_details = []
for topic in topic_info['Topic']:
    if topic != -1:  # Excluding the outlier topic
        words = topic_model.get_topic(topic)
        docs = topic_model.get_representative_docs(topic)
        topic_details.append({
            'Topic': topic,
            'Name': topic_info[topic_info['Topic'] == topic]['Name'].values[0],
            'Count': topic_info[topic_info['Topic'] == topic]['Count'].values[0],
            'Top Words': ', '.join([word for word, _ in words[:10]]),
            'Representative Docs': docs[:3]  # Getting the first 3 representative documents
        })

# Converting to DataFrame for easier viewing
df_topic_details = pd.DataFrame(topic_details)
print(df_topic_details.to_string())
